In [1]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
"""
=================================================================
Active Learning für MNIST - Robuste Version mit statistischer Analyse
=================================================================
Professionelles Skript für MNIST-Experimente mit fairem Vergleich
zwischen aktivem Lernen und Random Sampling als Baseline.

Verwendet den KOMPLETTEN MNIST-Datensatz (60.000 Trainingsbilder)
mit Budget-Stufen von 20%, 40%, 60%, 80% und 100%.

Version: 7.2 - Mit Label-Einsparungs-Analyse und Jupyter-Kompatibilität
            
Klassifikatoren:
- Naive Bayes
- Random Forest
- Logistic Regression
- SVM
- CNN

Query-Strategien:
- Random Sampling (Baseline)
- Entropy Sampling
- Margin Sampling
- Least Confidence

Statistische Analyse:
- Wilcoxon Signed-Rank Test
- Cliff's Delta Effektstärke
- Bonferroni-Korrektur für multiple Vergleiche
"""

import os
import sys
import time
import logging
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

# Matplotlib Backend setzen bevor pyplot importiert wird
import matplotlib
matplotlib.use('Agg')  # Für Server ohne GUI
import matplotlib.pyplot as plt

# Seaborn mit Fehlerbehandlung
try:
    import seaborn as sns
    # Prüfe ob der Style verfügbar ist
    try:
        plt.style.use('seaborn-v0_8-whitegrid')
    except:
        try:
            plt.style.use('seaborn-whitegrid')
        except:
            plt.style.use('ggplot')
except ImportError:
    print("Warnung: Seaborn nicht installiert. Verwende Standard-Matplotlib.")
    sns = None

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms

from sklearn.metrics import accuracy_score, f1_score, classification_report
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
import sklearn

# Statistische Tests
import scipy
from scipy import stats
from scipy.stats import wilcoxon
import itertools

# Excel-Export
try:
    import openpyxl
    EXCEL_AVAILABLE = True
except ImportError:
    print("Warnung: openpyxl nicht installiert. Excel-Export wird deaktiviert.")
    EXCEL_AVAILABLE = False

# SSL-Fehler beim Download verhindern
import ssl
try:
    ssl._create_default_https_context = ssl._create_unverified_context
except:
    pass

# -------------------------------------------------------------------------------
# Reproduzierbarkeit
# -------------------------------------------------------------------------------
SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

# -------------------------------------------------------------------------------
# Logging konfigurieren mit UTF-8 Encoding
# -------------------------------------------------------------------------------
# Erstelle Log-Verzeichnis
log_dir = "logs"
if not os.path.exists(log_dir):
    os.makedirs(log_dir)

# Logging Setup mit UTF-8 Encoding für Windows
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s [%(levelname)s] %(message)s",
    datefmt="%H:%M:%S",
    handlers=[
        logging.FileHandler(
            os.path.join(log_dir, f"active_learning_{time.strftime('%Y%m%d_%H%M%S')}.log"),
            encoding='utf-8'
        ),
        logging.StreamHandler(sys.stdout)
    ]
)
logger = logging.getLogger(__name__)

# Set encoding for stdout to handle Unicode - nur wenn möglich
if sys.platform == 'win32':
    # Prüfe ob wir in einer Jupyter/IPython Umgebung sind
    try:
        get_ipython()
        # In Jupyter/IPython - keine Änderung nötig
        logger.info("Jupyter/IPython Umgebung erkannt - UTF-8 Handling bereits aktiv")
    except NameError:
        # Normales Python - versuche UTF-8 zu setzen
        try:
            import io
            if hasattr(sys.stdout, 'buffer'):
                sys.stdout = io.TextIOWrapper(sys.stdout.buffer, encoding='utf-8')
                sys.stderr = io.TextIOWrapper(sys.stderr.buffer, encoding='utf-8')
        except Exception as e:
            logger.warning(f"Konnte UTF-8 Encoding nicht setzen: {e}")

# -------------------------------------------------------------------------------
# Konfiguration
# -------------------------------------------------------------------------------
BUDGET_PERCENTAGES = [0.2, 0.4, 0.6, 0.8, 1.0]  # 20%, 40%, 60%, 80%, 100%
BATCH_SIZE = 500  # Größere Batches für effizienteres Training
N_RUNS = 3  # Erhöht von 3 auf 5 für bessere statistische Aussagekraft
INITIAL_PERCENTAGE = 0.01  # 1% initial labeling
SIGNIFICANCE_LEVEL = 0.05  # Für statistische Tests

# Erstelle Output-Verzeichnisse
output_dirs = ["plots", "results", "reports"]
for dir_name in output_dirs:
    if not os.path.exists(dir_name):
        os.makedirs(dir_name)
        logger.info(f"Erstellt Verzeichnis: {dir_name}")

# -------------------------------------------------------------------------------
# 1) MNIST laden - VOLLSTÄNDIGER DATENSATZ
# -------------------------------------------------------------------------------
def load_mnist_data():
    """
    Lädt den VOLLSTÄNDIGEN MNIST-Datensatz (60.000 Trainingsbilder).
    Gibt auch eine flache Version für sklearn-Klassifikatoren zurück.
    """
    logger.info("Lade vollständigen MNIST-Datensatz...")
    
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
    ])
    
    # Erstelle Datenverzeichnis
    data_dir = './data'
    if not os.path.exists(data_dir):
        os.makedirs(data_dir)
    
    try:
        train_dataset = torchvision.datasets.MNIST(
            root=data_dir, train=True, download=True, transform=transform
        )
        test_dataset = torchvision.datasets.MNIST(
            root=data_dir, train=False, download=True, transform=transform
        )
    except Exception as e:
        logger.error(f"Fehler beim Laden des MNIST-Datensatzes: {e}")
        logger.info("Versuche alternativen Download...")
        
        # Alternative: Manuelle Download-URL setzen
        try:
            torchvision.datasets.MNIST.resources = [
                ('https://ossci-datasets.s3.amazonaws.com/mnist/train-images-idx3-ubyte.gz', 'f68b3c2dcbeaaa9fbdd348bbdeb94873'),
                ('https://ossci-datasets.s3.amazonaws.com/mnist/train-labels-idx1-ubyte.gz', 'd53e105ee54ea40749a09fcbcd1e9432'),
                ('https://ossci-datasets.s3.amazonaws.com/mnist/t10k-images-idx3-ubyte.gz', '9fb629c4189551a2d022fa330f9573f3'),
                ('https://ossci-datasets.s3.amazonaws.com/mnist/t10k-labels-idx1-ubyte.gz', 'ec29112dd5afa0611ce80d1b7f02629c')
            ]
            
            train_dataset = torchvision.datasets.MNIST(
                root=data_dir, train=True, download=True, transform=transform
            )
            test_dataset = torchvision.datasets.MNIST(
                root=data_dir, train=False, download=True, transform=transform
            )
        except Exception as e2:
            logger.error(f"Auch alternativer Download fehlgeschlagen: {e2}")
            raise RuntimeError("MNIST-Datensatz konnte nicht geladen werden. Bitte überprüfen Sie Ihre Internetverbindung.")
    
    # Konvertiere zu numpy arrays mit Fehlerbehandlung
    try:
        train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=len(train_dataset), shuffle=False)
        test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=len(test_dataset), shuffle=False)
        
        X_train, y_train = next(iter(train_loader))
        X_test, y_test = next(iter(test_loader))
    except Exception as e:
        logger.error(f"Fehler beim Konvertieren der Daten: {e}")
        raise
    
    # Für CNN (4D: batch, channel, height, width)
    X_train_cnn = X_train.numpy()
    X_test_cnn = X_test.numpy()
    
    # Für sklearn (2D: batch, features)
    X_train_flat = X_train.view(X_train.size(0), -1).numpy()
    X_test_flat = X_test.view(X_test.size(0), -1).numpy()
    
    y_train = y_train.numpy()
    y_test = y_test.numpy()
    
    # Validierung der Daten
    assert X_train_cnn.shape[0] == 60000, f"Unerwartete Anzahl von Trainingsbildern: {X_train_cnn.shape[0]}"
    assert X_test_cnn.shape[0] == 10000, f"Unerwartete Anzahl von Testbildern: {X_test_cnn.shape[0]}"
    assert len(np.unique(y_train)) == 10, f"Unerwartete Anzahl von Klassen: {len(np.unique(y_train))}"
    
    logger.info(f"[ok] Datensatz geladen: {len(X_train):,} Trainingsbilder, {len(X_test):,} Testbilder")
    logger.info(f"  CNN Format: {X_train_cnn.shape} | Flat Format: {X_train_flat.shape}")
    logger.info(f"  Klassen: {len(np.unique(y_train))}")
    logger.info(f"  Speicherbedarf: {(X_train_cnn.nbytes + X_train_flat.nbytes) / 1024**2:.1f} MB")
    
    return X_train_cnn, X_train_flat, y_train, X_test_cnn, X_test_flat, y_test

# -------------------------------------------------------------------------------
# 2) CNN Modell
# -------------------------------------------------------------------------------
class OptimizedCNN(nn.Module):
    """
    Optimierte CNN-Architektur für MNIST mit besserer Performance.
    """
    def __init__(self, num_classes=10):
        super(OptimizedCNN, self).__init__()
        
        # Feature extraction layers
        self.features = nn.Sequential(
            # Conv Block 1
            nn.Conv2d(1, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            nn.Conv2d(32, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2),
            
            # Conv Block 2
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2),
        )
        
        # Classifier
        self.classifier = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(64 * 7 * 7, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(256, num_classes)
        )
        
        # Device handling mit Fehlerbehandlung
        if torch.cuda.is_available():
            try:
                self.device = torch.device('cuda')
                # Test ob CUDA wirklich funktioniert
                test_tensor = torch.zeros(1).cuda()
                del test_tensor
            except:
                logger.warning("CUDA verfügbar aber nicht nutzbar. Verwende CPU.")
                self.device = torch.device('cpu')
        else:
            self.device = torch.device('cpu')
        
        self.to(self.device)
        
        # Initialize weights
        self._initialize_weights()

    def _initialize_weights(self):
        """Initialize network weights."""
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d) or isinstance(m, nn.BatchNorm1d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, 0, 0.01)
                nn.init.constant_(m.bias, 0)

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

    def fit(self, X_np, y_np, epochs=5, lr=1e-3, batch_size=256, verbose=False):
        """
        Trainiert das CNN mit optimierten Hyperparametern.
        """
        self.train()
        
        # Hyperparameter-Anpassung basierend auf Datensatzgröße
        if len(X_np) < 1000:
            batch_size = min(32, len(X_np))
            lr = lr * 0.1
        
        optimizer = optim.AdamW(self.parameters(), lr=lr, weight_decay=1e-4)
        scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=epochs)
        loss_fn = nn.CrossEntropyLoss()
        
        # Create dataset
        try:
            dataset = torch.utils.data.TensorDataset(
                torch.from_numpy(X_np).float(),
                torch.from_numpy(y_np).long()
            )
        except Exception as e:
            logger.error(f"Fehler beim Erstellen des Datasets: {e}")
            raise
        
        # DataLoader mit optimierten Settings
        loader = torch.utils.data.DataLoader(
            dataset, 
            batch_size=batch_size, 
            shuffle=True,
            num_workers=0,  # Immer 0 für Kompatibilität
            pin_memory=(self.device.type == 'cuda'),
            drop_last=False
        )
        
        # Training loop mit Fehlerbehandlung
        try:
            for epoch in range(epochs):
                total_loss = 0.0
                batch_count = 0
                
                for xb, yb in loader:
                    xb, yb = xb.to(self.device), yb.to(self.device)
                    
                    optimizer.zero_grad()
                    outputs = self(xb)
                    loss = loss_fn(outputs, yb)
                    
                    # Gradient clipping zur Stabilität
                    loss.backward()
                    torch.nn.utils.clip_grad_norm_(self.parameters(), max_norm=1.0)
                    
                    optimizer.step()
                    
                    total_loss += loss.item()
                    batch_count += 1
                
                scheduler.step()
                
                if verbose and (epoch + 1) % 2 == 0:
                    avg_loss = total_loss / max(batch_count, 1)
                    logger.info(f"    Epoch {epoch+1}/{epochs}, Loss: {avg_loss:.4f}")
                    
        except Exception as e:
            logger.error(f"Fehler während des Trainings: {e}")
            raise
        
        return self

    def predict_proba(self, X_np, batch_size=1024):
        """
        Gibt Wahrscheinlichkeiten für große Datenmengen zurück.
        """
        self.eval()
        probs = []
        
        # Anpassung der Batch-Größe bei wenig Speicher
        if self.device.type == 'cuda':
            try:
                free_memory = torch.cuda.get_device_properties(0).total_memory - torch.cuda.memory_allocated()
                if free_memory < 1024**3:  # Weniger als 1GB frei
                    batch_size = 256
            except:
                pass
        
        with torch.no_grad():
            try:
                for i in range(0, len(X_np), batch_size):
                    batch = torch.from_numpy(X_np[i:i+batch_size]).float().to(self.device)
                    logits = self(batch)
                    batch_probs = F.softmax(logits, dim=1)
                    probs.append(batch_probs.cpu().numpy())
                    
                    # Speicher freigeben
                    del batch, logits, batch_probs
                    if self.device.type == 'cuda':
                        torch.cuda.empty_cache()
                        
            except Exception as e:
                logger.error(f"Fehler bei predict_proba: {e}")
                raise
        
        return np.vstack(probs) if probs else np.array([])

    def predict(self, X_np, batch_size=1024):
        """
        Gibt Vorhersagen zurück.
        """
        probs = self.predict_proba(X_np, batch_size)
        return np.argmax(probs, axis=1) if len(probs) > 0 else np.array([])

# -------------------------------------------------------------------------------
# 3) Wrapper für sklearn-Klassifikatoren
# -------------------------------------------------------------------------------
class SklearnWrapper:
    """
    Wrapper-Klasse für sklearn-Klassifikatoren mit CNN-ähnlicher API.
    """
    def __init__(self, classifier, scaler=None):
        self.classifier = classifier
        self.scaler = scaler
        self.is_fitted = False
        
    def fit(self, X_np, y_np, **kwargs):
        """Trainiert den Klassifikator mit Fehlerbehandlung."""
        try:
            if self.scaler is not None:
                X_scaled = self.scaler.fit_transform(X_np)
            else:
                X_scaled = X_np
            
            self.classifier.fit(X_scaled, y_np)
            self.is_fitted = True
            
        except Exception as e:
            logger.error(f"Fehler beim Training des sklearn-Modells: {e}")
            raise
            
        return self
    
    def predict_proba(self, X_np):
        """Gibt Wahrscheinlichkeiten zurueck mit Fehlerbehandlung."""
        if not self.is_fitted:
            raise RuntimeError("Modell wurde noch nicht trainiert!")
        
        try:
            if self.scaler is not None:
                X_scaled = self.scaler.transform(X_np)
            else:
                X_scaled = X_np
                
            if hasattr(self.classifier, 'predict_proba'):
                return self.classifier.predict_proba(X_scaled)
            else:
                # Für SVM mit probability=False oder andere Klassifikatoren
                if hasattr(self.classifier, 'decision_function'):
                    decision = self.classifier.decision_function(X_scaled)
                    
                    # Multi-class Fall
                    if len(decision.shape) == 2:
                        # Softmax auf decision values
                        exp_decision = np.exp(decision - np.max(decision, axis=1, keepdims=True))
                        probs = exp_decision / np.sum(exp_decision, axis=1, keepdims=True)
                    else:
                        # Binary Fall - konvertiere zu 2-Klassen-Wahrscheinlichkeiten
                        probs = np.zeros((len(decision), 2))
                        probs[:, 1] = 1 / (1 + np.exp(-decision))
                        probs[:, 0] = 1 - probs[:, 1]
                    return probs
                else:
                    # Fallback: One-hot encoding der Vorhersagen
                    predictions = self.classifier.predict(X_scaled)
                    n_classes = len(np.unique(predictions))
                    probs = np.zeros((len(predictions), n_classes))
                    for i, pred in enumerate(predictions):
                        probs[i, int(pred)] = 1.0
                    return probs
                    
        except Exception as e:
            logger.error(f"Fehler bei predict_proba: {e}")
            raise
    
    def predict(self, X_np):
        """Gibt Vorhersagen zurück mit Fehlerbehandlung."""
        if not self.is_fitted:
            raise RuntimeError("Modell wurde noch nicht trainiert!")
            
        try:
            if self.scaler is not None:
                X_scaled = self.scaler.transform(X_np)
            else:
                X_scaled = X_np
            return self.classifier.predict(X_scaled)
            
        except Exception as e:
            logger.error(f"Fehler bei predict: {e}")
            raise

# -------------------------------------------------------------------------------
# 4) Klassifikator-Factory
# -------------------------------------------------------------------------------
def create_classifier(classifier_name, use_cnn_format=False):
    """
    Erstellt einen Klassifikator basierend auf dem Namen.
    
    Args:
        classifier_name: Name des Klassifikators
        use_cnn_format: True für CNN (4D), False für sklearn (2D)
    
    Returns:
        Klassifikator-Objekt mit einheitlicher API
    """
    try:
        if classifier_name == 'CNN':
            return OptimizedCNN()
        
        elif classifier_name == 'Naive Bayes':
            return SklearnWrapper(GaussianNB())
        
        elif classifier_name == 'Random Forest':
            # Angepasste Parameter für bessere Performance
            n_jobs = min(os.cpu_count() - 1, -1) if os.cpu_count() else -1
            return SklearnWrapper(
                RandomForestClassifier(
                    n_estimators=100,
                    max_depth=None,
                    min_samples_split=2,
                    min_samples_leaf=1,
                    n_jobs=n_jobs,
                    random_state=SEED,
                    verbose=0
                )
            )
        
        elif classifier_name == 'Logistic Regression':
            return SklearnWrapper(
                LogisticRegression(
                    max_iter=1000,
                    solver='saga',
                    multi_class='multinomial',
                    n_jobs=-1,
                    random_state=SEED,
                    verbose=0,
        		    penalty='l2',  
        	        C=0.1          	
                ),
                scaler=StandardScaler()
            )
        
        elif classifier_name == 'SVM':
            return SklearnWrapper(
                SVC(
                    kernel='rbf',
                    gamma='scale',
                    decision_function_shape='ovr',
                    probability=True,
                    cache_size=500,  # Mehr Cache für bessere Performance
                    random_state=SEED,
                    verbose=False
                ),
                scaler=StandardScaler()
            )
        
        else:
            raise ValueError(f"Unbekannter Klassifikator: {classifier_name}")
            
    except Exception as e:
        logger.error(f"Fehler beim Erstellen des Klassifikators {classifier_name}: {e}")
        raise

# -------------------------------------------------------------------------------
# 5) Query-Strategien mit Fehlerbehandlung
# -------------------------------------------------------------------------------
def entropy_sampling(model, X_pool, n_instances=1):
    """
    Waehlt Samples mit hoechster Entropie aus.
    H(x) = -Σ p(y|x) * log(p(y|x))
    """
    try:
        probs = model.predict_proba(X_pool)
        
        # Kleine Konstante hinzufügen um log(0) zu vermeiden
        epsilon = 1e-10
        probs = np.clip(probs, epsilon, 1.0 - epsilon)
        
        # Entropie berechnen
        entropies = -np.sum(probs * np.log(probs), axis=1)
        
        # Sicherstellen, dass wir nicht mehr Samples anfordern als verfügbar
        n_instances = min(n_instances, len(X_pool))
        
        # Indizes mit höchster Entropie
        return np.argsort(entropies)[-n_instances:]
        
    except Exception as e:
        logger.error(f"Fehler bei Entropy Sampling: {e}")
        # Fallback zu Random Sampling
        return random_sampling(model, X_pool, n_instances)

def margin_sampling(model, X_pool, n_instances=1):
    """
    Wählt Samples mit kleinstem Margin zwischen Top-2 Klassen.
    margin = P(y1|x) - P(y2|x)
    """
    try:
        probs = model.predict_proba(X_pool)
        
        # Sortiere Wahrscheinlichkeiten
        sorted_probs = np.sort(probs, axis=1)
        
        # Berechne Margin
        if sorted_probs.shape[1] >= 2:
            margins = sorted_probs[:, -1] - sorted_probs[:, -2]
        else:
            # Falls nur eine Klasse, verwende 1 - max_prob als Margin
            margins = 1.0 - sorted_probs[:, -1]
        
        # Sicherstellen, dass wir nicht mehr Samples anfordern als verfügbar
        n_instances = min(n_instances, len(X_pool))
        
        # Indizes mit kleinstem Margin
        return np.argsort(margins)[:n_instances]
        
    except Exception as e:
        logger.error(f"Fehler bei Margin Sampling: {e}")
        # Fallback zu Random Sampling
        return random_sampling(model, X_pool, n_instances)

def least_confidence_sampling(model, X_pool, n_instances=1):
    """
    Wählt Samples mit geringster Konfidenz.
    confidence = max P(y|x)
    """
    try:
        probs = model.predict_proba(X_pool)
        
        # Maximum-Wahrscheinlichkeit als Konfidenz
        confidences = np.max(probs, axis=1)
        
        # Sicherstellen, dass wir nicht mehr Samples anfordern als verfügbar
        n_instances = min(n_instances, len(X_pool))
        
        # Indizes mit geringster Konfidenz
        return np.argsort(confidences)[:n_instances]
        
    except Exception as e:
        logger.error(f"Fehler bei Least Confidence Sampling: {e}")
        # Fallback zu Random Sampling
        return random_sampling(model, X_pool, n_instances)

def random_sampling(model, X_pool, n_instances=1):
    """
    Zufällige Auswahl (Baseline).
    """
    try:
        # Sicherstellen, dass wir nicht mehr Samples anfordern als verfügbar
        n_instances = min(n_instances, len(X_pool))
        
        if n_instances <= 0:
            return np.array([], dtype=int)
            
        return np.random.choice(len(X_pool), size=n_instances, replace=False)
        
    except Exception as e:
        logger.error(f"Fehler bei Random Sampling: {e}")
        # Notfall-Fallback
        return np.arange(min(n_instances, len(X_pool)))

# -------------------------------------------------------------------------------
# 6) Statistische Analyse Funktionen mit Fehlerbehandlung
# -------------------------------------------------------------------------------
def cliffs_delta(x, y):
    """
    Berechnet Cliff's Delta als nicht-parametrisches Effektstaerkemaß.
    
    Interpretation:
    |d| < 0.147 "negligible"
    |d| < 0.33  "small" 
    |d| < 0.474 "medium"
    |d| >= 0.474 "large"
    """
    try:
        nx = len(x)
        ny = len(y)
        
        if nx == 0 or ny == 0:
            return 0.0
        
        # Konvertiere zu numpy arrays falls nötig
        x = np.asarray(x)
        y = np.asarray(y)
        
        # Berechne die Anzahl der Paare, wo x[i] > y[j]
        greater = 0
        less = 0
        
        # Vektorisierte Berechnung für bessere Performance
        for xi in x:
            greater += np.sum(xi > y)
            less += np.sum(xi < y)
        
        # Cliff's Delta
        d = (greater - less) / (nx * ny)
        
        # Sicherstellen, dass d im Bereich [-1, 1] liegt
        d = np.clip(d, -1.0, 1.0)
        
        return d
        
    except Exception as e:
        logger.error(f"Fehler bei Cliff's Delta Berechnung: {e}")
        return 0.0

def interpret_cliffs_delta(d):
    """
    Interpretiert die Effektstärke nach Cliff's Delta.
    """
    try:
        abs_d = abs(float(d))
        if abs_d < 0.147:
            return "negligible"
        elif abs_d < 0.33:
            return "small"
        elif abs_d < 0.474:
            return "medium"
        else:
            return "large"
    except:
        return "unknown"

def perform_statistical_analysis(results_df, metric='accuracy'):
    """
    Fuehrt statistische Analyse durch: Wilcoxon Signed-Rank Test mit Bonferroni-Korrektur.
    """
    statistical_results = []
    
    try:
        classifiers = results_df['classifier'].unique()
        strategies = results_df['strategy'].unique()
        budget_levels = results_df['budget_pct'].unique()
        
        for classifier in classifiers:
            for budget_pct in budget_levels:
                # Hole Random Sampling als Baseline
                baseline_data = results_df[
                    (results_df['classifier'] == classifier) & 
                    (results_df['strategy'] == 'Random Sampling') & 
                    (results_df['budget_pct'] == budget_pct)
                ][metric].values
                
                # Vergleiche mit anderen Strategien
                for strategy in strategies:
                    if strategy == 'Random Sampling':
                        continue
                        
                    strategy_data = results_df[
                        (results_df['classifier'] == classifier) & 
                        (results_df['strategy'] == strategy) & 
                        (results_df['budget_pct'] == budget_pct)
                    ][metric].values
                    
                    # Mindestens N_RUNS Datenpunkte für statistische Tests
                    if len(baseline_data) >= N_RUNS and len(strategy_data) >= N_RUNS:
                        # Wilcoxon Signed-Rank Test
                        try:
                            # Prüfe ob alle Werte gleich sind
                            if np.allclose(strategy_data, baseline_data):
                                statistic, p_value = 0.0, 1.0
                            else:
                                statistic, p_value = wilcoxon(
                                    strategy_data, baseline_data, 
                                    alternative='greater',
                                    zero_method='zsplit'
                                )
                        except Exception as e:
                            logger.warning(f"Wilcoxon Test fehlgeschlagen für {classifier}-{strategy}: {e}")
                            statistic, p_value = 0.0, 1.0
                        
                        # Effektstärke
                        effect_size = cliffs_delta(strategy_data, baseline_data)
                        effect_interpretation = interpret_cliffs_delta(effect_size)
                        
                        # Mittelwerte und Standardabweichungen
                        baseline_mean = np.mean(baseline_data) if len(baseline_data) > 0 else 0
                        baseline_std = np.std(baseline_data) if len(baseline_data) > 0 else 0
                        strategy_mean = np.mean(strategy_data) if len(strategy_data) > 0 else 0
                        strategy_std = np.std(strategy_data) if len(strategy_data) > 0 else 0
                        
                        # Verbesserung berechnen mit Division-by-Zero-Schutz
                        improvement = strategy_mean - baseline_mean
                        improvement_pct = ((improvement / baseline_mean) * 100) if baseline_mean > 0 else 0
                        
                        statistical_results.append({
                            'classifier': classifier,
                            'budget_pct': budget_pct,
                            'strategy': strategy,
                            'baseline_mean': baseline_mean,
                            'baseline_std': baseline_std,
                            'strategy_mean': strategy_mean,
                            'strategy_std': strategy_std,
                            'improvement': improvement,
                            'improvement_pct': improvement_pct,
                            'wilcoxon_statistic': float(statistic),
                            'p_value': float(p_value),
                            'cliffs_delta': float(effect_size),
                            'effect_size': effect_interpretation,
                            'n_samples': len(strategy_data)
                        })
        
        # Konvertiere zu DataFrame
        stat_df = pd.DataFrame(statistical_results)
        
        if len(stat_df) > 0:
            # Bonferroni-Korrektur für multiple Vergleiche
            n_comparisons = len(stat_df)
            stat_df['p_value_corrected'] = np.minimum(stat_df['p_value'] * n_comparisons, 1.0)
            stat_df['significant'] = stat_df['p_value_corrected'] < SIGNIFICANCE_LEVEL
        else:
            logger.warning("Keine statistischen Ergebnisse generiert!")
            
        return stat_df
        
    except Exception as e:
        logger.error(f"Fehler bei der statistischen Analyse: {e}")
        return pd.DataFrame()  # Leerer DataFrame als Fallback

# -------------------------------------------------------------------------------
# 7) Active Learning Hauptfunktion mit Fehlerbehandlung
# -------------------------------------------------------------------------------
def run_active_learning_experiment(X_train_cnn, X_train_flat, y_train, 
                                 X_test_cnn, X_test_flat, y_test,
                                 classifier_name, strategy_name, strategy_func,
                                 budget_percentages, batch_size=500):
    """
    Führt ein Active Learning Experiment durch mit umfassender Fehlerbehandlung.
    """
    results = []
    n_total = len(y_train)
    
    # Wähle richtiges Datenformat
    use_cnn = (classifier_name == 'CNN')
    X_train = X_train_cnn if use_cnn else X_train_flat
    X_test = X_test_cnn if use_cnn else X_test_flat
    
    for budget_pct in budget_percentages:
        n_budget = int(budget_pct * n_total)
        
        logger.info(f"\n{classifier_name} + {strategy_name} - Budget: {budget_pct:.0%} ({n_budget:,} Samples)")
        
        for run in range(N_RUNS):
            logger.info(f"  Run {run+1}/{N_RUNS}")
            
            try:
                # Set seed for reproducibility
                np.random.seed(SEED + run)
                if use_cnn:
                    torch.manual_seed(SEED + run)
                    if torch.cuda.is_available():
                        torch.cuda.manual_seed(SEED + run)
                
                # Initialisierung
                pool_indices = np.arange(n_total)
                labeled_indices = []
                
                # Initiale zufällige Auswahl
                n_initial = max(100, int(INITIAL_PERCENTAGE * n_total))
                n_initial = min(n_initial, len(pool_indices))  # Sicherstellen dass genug Samples da sind
                
                initial_indices = np.random.choice(pool_indices, size=n_initial, replace=False)
                labeled_indices = list(initial_indices)
                pool_indices = np.setdiff1d(pool_indices, labeled_indices)
                
                # Tracking
                accuracies = []
                n_labeled_list = []
                query_times = []
                train_times = []
                
                while len(labeled_indices) < n_budget and len(pool_indices) > 0:
                    start_time = time.time()
                    
                    # Modell erstellen und trainieren
                    model = create_classifier(classifier_name, use_cnn)
                    
                    train_start = time.time()
                    
                    # Training mit Fehlerbehandlung
                    try:
                        if use_cnn:
                            model.fit(X_train[labeled_indices], y_train[labeled_indices], 
                                     epochs=5, verbose=False)
                        else:
                            model.fit(X_train[labeled_indices], y_train[labeled_indices])
                    except Exception as e:
                        logger.error(f"Fehler beim Training in Run {run+1}: {e}")
                        # Skip diesen Durchlauf
                        break
                    
                    train_time = time.time() - train_start
                    train_times.append(train_time)
                    
                    # Evaluation
                    try:
                        y_pred = model.predict(X_test)
                        acc = accuracy_score(y_test, y_pred)
                    except Exception as e:
                        logger.error(f"Fehler bei der Evaluation in Run {run+1}: {e}")
                        acc = 0.0
                    
                    accuracies.append(acc)
                    n_labeled_list.append(len(labeled_indices))
                    
                    # Nächste Batch auswählen
                    n_query = min(batch_size, n_budget - len(labeled_indices), len(pool_indices))
                    if n_query <= 0:
                        break
                    
                    # Query mit Zeitmessung und Fehlerbehandlung
                    query_start = time.time()
                    try:
                        query_indices = strategy_func(model, X_train[pool_indices], n_query)
                    except Exception as e:
                        logger.error(f"Fehler bei Query-Strategie {strategy_name}: {e}")
                        # Fallback zu Random Sampling
                        query_indices = random_sampling(model, X_train[pool_indices], n_query)
                    
                    query_time = time.time() - query_start
                    query_times.append(query_time)
                    
                    # Validierung der Query-Indizes
                    query_indices = np.asarray(query_indices)
                    query_indices = query_indices[query_indices < len(pool_indices)]  # Entferne ungültige Indizes
                    
                    if len(query_indices) == 0:
                        logger.warning(f"Keine gueltigen Query-Indizes in Run {run+1}")
                        break
                    
                    selected_indices = pool_indices[query_indices]
                    
                    # Update
                    labeled_indices.extend(selected_indices)
                    pool_indices = np.setdiff1d(pool_indices, selected_indices)
                    
                    # Progress logging - Unicode-Fix: Verwende -> statt →
                    if len(labeled_indices) % 5000 == 0 or len(labeled_indices) == n_budget:
                        logger.info(f"    {len(labeled_indices):,} labeled -> Accuracy: {acc:.4f} "
                                  f"(Train: {train_time:.1f}s, Query: {query_time:.2f}s)")
                    
                    # Speicher freigeben bei CNN
                    if use_cnn and hasattr(model, 'device') and model.device.type == 'cuda':
                        torch.cuda.empty_cache()
                
                # Finale Evaluation mit mehr Training
                if len(labeled_indices) > 0:
                    try:
                        model = create_classifier(classifier_name, use_cnn)
                        
                        if use_cnn:
                            model.fit(X_train[labeled_indices], y_train[labeled_indices], 
                                     epochs=10, verbose=False)
                        else:
                            model.fit(X_train[labeled_indices], y_train[labeled_indices])
                        
                        y_pred = model.predict(X_test)
                        final_acc = accuracy_score(y_test, y_pred)
                        final_f1 = f1_score(y_test, y_pred, average='macro')
                        
                    except Exception as e:
                        logger.error(f"Fehler bei finaler Evaluation in Run {run+1}: {e}")
                        final_acc = acc if 'acc' in locals() else 0.0
                        final_f1 = 0.0
                    
                    results.append({
                        'classifier': classifier_name,
                        'strategy': strategy_name,
                        'budget_pct': budget_pct,
                        'run': run,
                        'n_labeled': len(labeled_indices),
                        'accuracy': final_acc,
                        'f1_score': final_f1,
                        'accuracies': accuracies,
                        'n_labeled_list': n_labeled_list,
                        'avg_query_time': np.mean(query_times) if query_times else 0,
                        'avg_train_time': np.mean(train_times) if train_times else 0
                    })
                    
                    # Unicode-Fix: Verwende -> statt →
                    logger.info(f"    Final: {len(labeled_indices):,} labeled -> "
                              f"Accuracy: {final_acc:.4f}, F1: {final_f1:.4f}")
                
                # Speicher freigeben
                if use_cnn and torch.cuda.is_available():
                    torch.cuda.empty_cache()
                    
            except Exception as e:
                logger.error(f"Unerwarteter Fehler in Run {run+1}: {e}")
                import traceback
                traceback.print_exc()
                continue
    
    return results

# -------------------------------------------------------------------------------
# 8) Visualisierung: Pro Klassifikator mit Signifikanzmarkierungen
# -------------------------------------------------------------------------------
def plot_per_classifier_with_significance(all_results, stat_results):
    """
    Erstellt eine Visualisierung pro Klassifikator mit Signifikanzmarkierungen.
    """
    try:
        # Style setzen mit Fallback
        try:
            plt.style.use('seaborn-v0_8-whitegrid')
        except:
            try:
                plt.style.use('seaborn-whitegrid')
            except:
                plt.style.use('ggplot')
        
        # Farben für Strategien
        strategy_colors = {
            'Random Sampling': '#808080',
            'Entropy Sampling': '#1f77b4',
            'Margin Sampling': '#ff7f0e',
            'Least Confidence': '#2ca02c'
        }
        
        # Klassifikatoren extrahieren
        classifiers = sorted(list(set(r['classifier'] for r in all_results)))
        
        # Eine Figure pro Klassifikator
        for classifier in classifiers:
            fig, axes = plt.subplots(1, len(BUDGET_PERCENTAGES), figsize=(20, 4))
            
            # Handle für einzelne Subplot
            if len(BUDGET_PERCENTAGES) == 1:
                axes = [axes]
                
            fig.suptitle(f'{classifier} - Active Learning Performance with Statistical Significance', 
                         fontsize=16, y=1.02)
            
            for budget_idx, budget_pct in enumerate(BUDGET_PERCENTAGES):
                ax = axes[budget_idx]
                
                # Daten für diesen Klassifikator und Budget
                for strategy, color in strategy_colors.items():
                    strategy_results = [r for r in all_results 
                                      if r['classifier'] == classifier 
                                      and r['strategy'] == strategy 
                                      and r['budget_pct'] == budget_pct]
                    
                    if strategy_results:
                        # Lernkurven aggregieren
                        max_samples = int(budget_pct * 60000)
                        x_common = np.linspace(100, max_samples, 100)
                        y_interpolated = []
                        
                        for r in strategy_results:
                            if len(r['n_labeled_list']) > 1:
                                try:
                                    y_interp = np.interp(x_common, r['n_labeled_list'], r['accuracies'])
                                    y_interpolated.append(y_interp)
                                except:
                                    logger.warning(f"Interpolation fehlgeschlagen für {classifier}-{strategy}")
                        
                        if y_interpolated:
                            y_mean = np.mean(y_interpolated, axis=0)
                            y_std = np.std(y_interpolated, axis=0)
                            
                            # Überprüfe Signifikanz
                            is_significant = False
                            effect_size = ""
                            if strategy != 'Random Sampling' and not stat_results.empty:
                                sig_data = stat_results[
                                    (stat_results['classifier'] == classifier) & 
                                    (stat_results['strategy'] == strategy) & 
                                    (stat_results['budget_pct'] == budget_pct)
                                ]
                                if not sig_data.empty:
                                    is_significant = sig_data.iloc[0]['significant']
                                    effect_size = sig_data.iloc[0]['effect_size']
                            
                            # Label mit Signifikanz
                            label = strategy
                            if is_significant:
                                label += f" *({effect_size})"
                            
                            # Plot mit Konfidenzintervall
                            ax.plot(x_common, y_mean, 
                                   label=label, 
                                   color=color, 
                                   linewidth=2.5,
                                   linestyle='-' if not is_significant or strategy == 'Random Sampling' else '--',
                                   marker='o' if strategy == 'Random Sampling' else None,
                                   markevery=10 if strategy == 'Random Sampling' else None,
                                   markersize=4)
                            
                            ax.fill_between(x_common, 
                                          y_mean - y_std, 
                                          y_mean + y_std, 
                                          color=color, 
                                          alpha=0.2)
                
                # Achsenbeschriftung
                ax.set_xlabel('Number of Labeled Samples', fontsize=11)
                ax.set_ylabel('Test Accuracy', fontsize=11)
                ax.set_title(f'Budget: {int(budget_pct*100)}%', fontsize=12)
                
                # Grid und Limits
                ax.grid(True, alpha=0.3)
                ax.set_ylim([0.75, 1.0])
                
                # X-Achse formatieren
                ax.ticklabel_format(style='plain', axis='x')
                ax.xaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'{int(x/1000)}k'))
                
                # Legende nur beim ersten Plot
                if budget_idx == 0:
                    ax.legend(loc='lower right', fontsize=9, framealpha=0.9)
            
            # Signifikanz-Erklärung
            fig.text(0.5, -0.05, 
                    '* = statistically significant (p < 0.05 with Bonferroni correction); ' +
                    'Effect size in parentheses (negligible/small/medium/large)',
                    ha='center', fontsize=10, style='italic')
            
            plt.tight_layout()
            
            # Speichern mit Fehlerbehandlung
            filename = f'plots/active_learning_{classifier.lower().replace(" ", "_")}_with_significance.png'
            try:
                plt.savefig(filename, dpi=300, bbox_inches='tight')
                logger.info(f"[ok] Visualisierung mit Signifikanz für {classifier} erstellt: {filename}")
            except Exception as e:
                logger.error(f"Fehler beim Speichern der Visualisierung für {classifier}: {e}")
                
            plt.close()
            
    except Exception as e:
        logger.error(f"Fehler bei plot_per_classifier_with_significance: {e}")
        import traceback
        traceback.print_exc()

# -------------------------------------------------------------------------------
# 9) Visualisierung: Statistische Zusammenfassung
# -------------------------------------------------------------------------------
def plot_statistical_summary(stat_results):
    """
    Erstellt eine Visualisierung der statistischen Ergebnisse.
    """
    try:
        # Style setzen
        try:
            plt.style.use('seaborn-v0_8-whitegrid')
        except:
            try:
                plt.style.use('seaborn-whitegrid')
            except:
                plt.style.use('ggplot')
        
        if stat_results.empty:
            logger.warning("Keine statistischen Ergebnisse zum Visualisieren!")
            return
            
        # Filtere nur signifikante Ergebnisse
        sig_results = stat_results[stat_results['significant']].copy() if 'significant' in stat_results.columns else pd.DataFrame()
        
        if sig_results.empty:
            logger.warning("Keine signifikanten Ergebnisse gefunden!")
            # Erstelle trotzdem eine Visualisierung mit allen Ergebnissen
            sig_results = stat_results.copy()
        
        # Figure mit Subplots
        fig, axes = plt.subplots(2, 2, figsize=(16, 12))
        fig.suptitle('Statistical Analysis Summary', fontsize=16)
        
        # 1. Heatmap der p-Werte
        try:
            ax1 = axes[0, 0]
            pivot_p = stat_results.pivot_table(
                values='p_value_corrected',
                index=['classifier', 'strategy'],
                columns='budget_pct',
                fill_value=1.0
            )
            
            if sns is not None and not pivot_p.empty:
                sns.heatmap(pivot_p, 
                            annot=True, 
                            fmt='.3f', 
                            cmap='RdYlGn_r',
                            vmin=0, 
                            vmax=0.1,
                            cbar_kws={'label': 'Corrected p-value'},
                            ax=ax1)
            else:
                ax1.text(0.5, 0.5, 'Heatmap nicht verfügbar', ha='center', va='center')
                
            ax1.set_title('Corrected p-values (Wilcoxon Signed-Rank Test)')
            ax1.set_xlabel('Budget (%)')
        except Exception as e:
            logger.warning(f"Fehler bei p-Wert Heatmap: {e}")
            axes[0, 0].text(0.5, 0.5, 'Fehler bei der Erstellung', ha='center', va='center')
        
        # 2. Heatmap der Effektstärken
        try:
            ax2 = axes[0, 1]
            pivot_effect = stat_results.pivot_table(
                values='cliffs_delta',
                index=['classifier', 'strategy'],
                columns='budget_pct',
                fill_value=0.0
            )
            
            if sns is not None and not pivot_effect.empty:
                sns.heatmap(pivot_effect, 
                            annot=True, 
                            fmt='.3f', 
                            cmap='coolwarm',
                            center=0,
                            vmin=-1, 
                            vmax=1,
                            cbar_kws={'label': "Cliff's Delta"},
                            ax=ax2)
            else:
                ax2.text(0.5, 0.5, 'Heatmap nicht verfügbar', ha='center', va='center')
                
            ax2.set_title("Effect Size (Cliff's Delta)")
            ax2.set_xlabel('Budget (%)')
        except Exception as e:
            logger.warning(f"Fehler bei Effektstärken Heatmap: {e}")
            axes[0, 1].text(0.5, 0.5, 'Fehler bei der Erstellung', ha='center', va='center')
        
        # 3. Anzahl signifikanter Verbesserungen pro Strategie
        try:
            ax3 = axes[1, 0]
            if not sig_results.empty and 'strategy' in sig_results.columns:
                sig_counts = sig_results.groupby('strategy').size().sort_values(ascending=False)
                sig_counts.plot(kind='bar', ax=ax3, color='steelblue')
                ax3.set_title('Number of Significant Improvements per Strategy')
                ax3.set_xlabel('Strategy')
                ax3.set_ylabel('Count')
                ax3.grid(axis='y', alpha=0.3)
            else:
                ax3.text(0.5, 0.5, 'Keine signifikanten Ergebnisse', ha='center', va='center')
        except Exception as e:
            logger.warning(f"Fehler bei Signifikanz-Barplot: {e}")
            axes[1, 0].text(0.5, 0.5, 'Fehler bei der Erstellung', ha='center', va='center')
        
        # 4. Durchschnittliche Effektstärke pro Klassifikator
        try:
            ax4 = axes[1, 1]
            if 'classifier' in stat_results.columns and 'cliffs_delta' in stat_results.columns:
                avg_effect = stat_results.groupby('classifier')['cliffs_delta'].agg(['mean', 'std'])
                avg_effect['mean'].plot(kind='bar', ax=ax4, yerr=avg_effect['std'], 
                                       capsize=5, color='darkorange')
                ax4.set_title("Average Effect Size per Classifier")
                ax4.set_xlabel('Classifier')
                ax4.set_ylabel("Mean Cliff's Delta")
                ax4.axhline(y=0, color='black', linestyle='-', linewidth=0.5)
                ax4.grid(axis='y', alpha=0.3)
            else:
                ax4.text(0.5, 0.5, 'Daten nicht verfügbar', ha='center', va='center')
        except Exception as e:
            logger.warning(f"Fehler bei Effektstärken-Barplot: {e}")
            axes[1, 1].text(0.5, 0.5, 'Fehler bei der Erstellung', ha='center', va='center')
        
        plt.tight_layout()
        
        # Speichern
        filename = 'plots/active_learning_statistical_summary.png'
        try:
            plt.savefig(filename, dpi=300, bbox_inches='tight')
            logger.info(f"[ok] Statistische Zusammenfassung erstellt: {filename}")
        except Exception as e:
            logger.error(f"Fehler beim Speichern der statistischen Zusammenfassung: {e}")
            
        plt.close()
        
    except Exception as e:
        logger.error(f"Fehler bei plot_statistical_summary: {e}")
        import traceback
        traceback.print_exc()

# -------------------------------------------------------------------------------
# 10) Detaillierte statistische Tabelle
# -------------------------------------------------------------------------------
def create_statistical_report(stat_results):
    """
    Erstellt einen detaillierten statistischen Bericht.
    """
    try:
        # Sortiere nach Effektstärke
        if not stat_results.empty and 'cliffs_delta' in stat_results.columns:
            stat_results_sorted = stat_results.sort_values('cliffs_delta', ascending=False)
        else:
            stat_results_sorted = stat_results
        
        # Erstelle formatierten Bericht
        report = []
        report.append("\n" + "="*100)
        report.append("DETAILLIERTER STATISTISCHER BERICHT")
        report.append("="*100)
        report.append(f"Signifikanzniveau: {SIGNIFICANCE_LEVEL} (mit Bonferroni-Korrektur)")
        report.append(f"Anzahl Runs pro Experiment: {N_RUNS}")
        report.append(f"Statistischer Test: Wilcoxon Signed-Rank Test")
        report.append(f"Effektstärkemaß: Cliff's Delta")
        report.append("\n")
        
        # Signifikante Ergebnisse
        if 'significant' in stat_results_sorted.columns:
            sig_results = stat_results_sorted[stat_results_sorted['significant']]
        else:
            sig_results = pd.DataFrame()
        
        if not sig_results.empty:
            report.append("SIGNIFIKANTE VERBESSERUNGEN GEGENÜBER RANDOM SAMPLING:")
            report.append("-"*100)
            report.append(f"{'Klassifikator':<20} {'Strategie':<20} {'Budget':<10} {'Verbesserung':<15} "
                         f"{'p-Wert':<12} {'Effekt':<15} {'Interpretation':<15}")
            report.append("-"*100)
            
            for _, row in sig_results.iterrows():
                report.append(f"{row['classifier']:<20} {row['strategy']:<20} "
                             f"{int(row['budget_pct']*100):>8}% "
                             f"{row['improvement_pct']:>13.2f}% "
                             f"{row['p_value_corrected']:>11.4f} "
                             f"{row['cliffs_delta']:>14.3f} "
                             f"{row['effect_size']:<15}")
        else:
            report.append("Keine signifikanten Verbesserungen gefunden!")
        
        # Zusammenfassung nach Strategie
        report.append("\n\nZUSAMMENFASSUNG NACH STRATEGIE:")
        report.append("-"*100)
        
        for strategy in ['Entropy Sampling', 'Margin Sampling', 'Least Confidence']:
            if 'strategy' in stat_results.columns:
                strategy_data = stat_results[stat_results['strategy'] == strategy]
                if not strategy_data.empty:
                    sig_count = strategy_data['significant'].sum() if 'significant' in strategy_data.columns else 0
                    avg_improvement = strategy_data['improvement_pct'].mean() if 'improvement_pct' in strategy_data.columns else 0
                    avg_effect = strategy_data['cliffs_delta'].mean() if 'cliffs_delta' in strategy_data.columns else 0
                    
                    report.append(f"\n{strategy}:")
                    report.append(f"  - Signifikante Verbesserungen: {sig_count}/{len(strategy_data)} "
                                 f"({sig_count/len(strategy_data)*100:.1f}%)")
                    report.append(f"  - Durchschnittliche Verbesserung: {avg_improvement:.2f}%")
                    report.append(f"  - Durchschnittliche Effektstärke: {avg_effect:.3f}")
        
        # Zusammenfassung nach Klassifikator
        report.append("\n\nZUSAMMENFASSUNG NACH KLASSIFIKATOR:")
        report.append("-"*100)
        
        if 'classifier' in stat_results.columns:
            for classifier in stat_results['classifier'].unique():
                classifier_data = stat_results[stat_results['classifier'] == classifier]
                sig_count = classifier_data['significant'].sum() if 'significant' in classifier_data.columns else 0
                
                report.append(f"\n{classifier}:")
                report.append(f"  - Signifikante Verbesserungen: {sig_count}/{len(classifier_data)} "
                             f"({sig_count/len(classifier_data)*100:.1f}%)")
                
                if sig_count > 0 and 'cliffs_delta' in classifier_data.columns:
                    best_strategy = classifier_data.loc[classifier_data['cliffs_delta'].idxmax()]
                    report.append(f"  - Beste Strategie: {best_strategy['strategy']} "
                                 f"bei {int(best_strategy['budget_pct']*100)}% Budget "
                                 f"(Effekt: {best_strategy['cliffs_delta']:.3f})")
        
        report.append("\n" + "="*100)
        
        # Ausgabe
        report_text = "\n".join(report)
        print(report_text)
        
        # Speichern
        report_filename = 'reports/statistical_report.txt'
        try:
            with open(report_filename, 'w', encoding='utf-8') as f:
                f.write(report_text)
            logger.info(f"[ok] Statistischer Bericht gespeichert: {report_filename}")
        except Exception as e:
            logger.error(f"Fehler beim Speichern des Berichts: {e}")
        
        return report_text
        
    except Exception as e:
        logger.error(f"Fehler bei create_statistical_report: {e}")
        return "Fehler bei der Berichterstellung"

# -------------------------------------------------------------------------------
# 11) Visualisierung: Finale Vergleichsmatrix
# -------------------------------------------------------------------------------
def plot_final_comparison(all_results):
    """
    Erstellt eine Visualisierung, die zeigt, welche Kombination aus Klassifikator
    und Query-Strategie bei 100% Budget am besten abgeschnitten hat.
    """
    try:
        # Style setzen
        try:
            plt.style.use('seaborn-v0_8-whitegrid')
        except:
            try:
                plt.style.use('seaborn-whitegrid')
            except:
                plt.style.use('ggplot')
        
        # Nur Ergebnisse mit 100% Budget
        final_results = [r for r in all_results if r['budget_pct'] == 1.0]
        
        if not final_results:
            logger.warning("Keine Ergebnisse mit 100% Budget gefunden!")
            return
        
        # Aggregiere Ergebnisse
        summary_data = []
        classifiers = sorted(list(set(r['classifier'] for r in final_results)))
        strategies = ['Random Sampling', 'Entropy Sampling', 'Margin Sampling', 'Least Confidence']
        
        for classifier in classifiers:
            for strategy in strategies:
                results = [r for r in final_results 
                          if r['classifier'] == classifier and r['strategy'] == strategy]
                
                if results:
                    mean_acc = np.mean([r['accuracy'] for r in results])
                    std_acc = np.std([r['accuracy'] for r in results])
                    mean_f1 = np.mean([r['f1_score'] for r in results])
                    
                    summary_data.append({
                        'Classifier': classifier,
                        'Strategy': strategy,
                        'Accuracy': mean_acc,
                        'Accuracy_std': std_acc,
                        'F1-Score': mean_f1
                    })
        
        if not summary_data:
            logger.warning("Keine Daten für finale Vergleichsmatrix!")
            return
            
        # DataFrame erstellen
        df = pd.DataFrame(summary_data)
        
        # Pivot für Heatmap
        pivot_acc = df.pivot(index='Classifier', columns='Strategy', values='Accuracy')
        pivot_f1 = df.pivot(index='Classifier', columns='Strategy', values='F1-Score')
        
        # Figure mit zwei Subplots
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))
        
        # Heatmap für Accuracy
        if sns is not None and not pivot_acc.empty:
            sns.heatmap(pivot_acc, 
                        annot=True, 
                        fmt='.4f', 
                        cmap='RdYlGn', 
                        vmin=0.95, 
                        vmax=1.0,
                        cbar_kws={'label': 'Test Accuracy'},
                        ax=ax1)
        else:
            ax1.text(0.5, 0.5, 'Heatmap nicht verfügbar', ha='center', va='center')
            
        ax1.set_title('Test Accuracy at 100% Budget', fontsize=14)
        ax1.set_xlabel('Query Strategy', fontsize=12)
        ax1.set_ylabel('Classifier', fontsize=12)
        
        # Heatmap für F1-Score
        if sns is not None and not pivot_f1.empty:
            sns.heatmap(pivot_f1, 
                        annot=True, 
                        fmt='.4f', 
                        cmap='RdYlGn', 
                        vmin=0.95, 
                        vmax=1.0,
                        cbar_kws={'label': 'F1-Score (Macro)'},
                        ax=ax2)
        else:
            ax2.text(0.5, 0.5, 'Heatmap nicht verfügbar', ha='center', va='center')
            
        ax2.set_title('F1-Score at 100% Budget', fontsize=14)
        ax2.set_xlabel('Query Strategy', fontsize=12)
        ax2.set_ylabel('Classifier', fontsize=12)
        
        plt.suptitle('Final Performance Comparison - Full Dataset (100% Budget)', fontsize=16)
        plt.tight_layout()
        
        # Speichern
        filename = 'plots/active_learning_final_comparison.png'
        try:
            plt.savefig(filename, dpi=300, bbox_inches='tight')
            logger.info(f"[ok] Finale Vergleichsvisualisierung erstellt: {filename}")
        except Exception as e:
            logger.error(f"Fehler beim Speichern der finalen Vergleichsmatrix: {e}")
            
        plt.close()
        
        # Beste Kombination finden
        if not df.empty:
            best_idx = df['Accuracy'].idxmax()
            best_result = df.iloc[best_idx]
            
            print("\n" + "="*60)
            print("BESTE KOMBINATION BEI 100% BUDGET:")
            print("="*60)
            print(f"Klassifikator: {best_result['Classifier']}")
            print(f"Query-Strategie: {best_result['Strategy']}")
            print(f"Test Accuracy: {best_result['Accuracy']:.4f} (±{best_result['Accuracy_std']:.4f})")
            print(f"F1-Score: {best_result['F1-Score']:.4f}")
            print("="*60)
            
            # Top 5 Kombinationen
            print("\nTOP 5 KOMBINATIONEN:")
            print("-"*60)
            top5 = df.nlargest(5, 'Accuracy')[['Classifier', 'Strategy', 'Accuracy', 'F1-Score']]
            for idx, (_, row) in enumerate(top5.iterrows()):
                print(f"{idx+1}. {row['Classifier']} + {row['Strategy']}: "
                      f"Acc={row['Accuracy']:.4f}, F1={row['F1-Score']:.4f}")
        
    except Exception as e:
        logger.error(f"Fehler bei plot_final_comparison: {e}")
        import traceback
        traceback.print_exc()

# -------------------------------------------------------------------------------
# 12) Zusätzliche Analyse: Improvement over Random
# -------------------------------------------------------------------------------
def plot_improvement_analysis(all_results):
    """
    Zeigt die Verbesserung der Active Learning Strategien gegenüber Random Sampling.
    """
    try:
        # Style setzen
        try:
            plt.style.use('seaborn-v0_8-whitegrid')
        except:
            try:
                plt.style.use('seaborn-whitegrid')
            except:
                plt.style.use('ggplot')
        
        # Berechne Verbesserungen
        improvements = []
        classifiers = sorted(list(set(r['classifier'] for r in all_results)))
        strategies = ['Entropy Sampling', 'Margin Sampling', 'Least Confidence']
        
        for classifier in classifiers:
            for strategy in strategies:
                for budget_pct in BUDGET_PERCENTAGES:
                    # Random Sampling Baseline
                    random_results = [r for r in all_results 
                                    if r['classifier'] == classifier 
                                    and r['strategy'] == 'Random Sampling' 
                                    and r['budget_pct'] == budget_pct]
                    
                    # Active Learning Strategy
                    strategy_results = [r for r in all_results 
                                      if r['classifier'] == classifier 
                                      and r['strategy'] == strategy 
                                      and r['budget_pct'] == budget_pct]
                    
                    if random_results and strategy_results:
                        random_acc = np.mean([r['accuracy'] for r in random_results])
                        strategy_acc = np.mean([r['accuracy'] for r in strategy_results])
                        
                        # Prozentuale Verbesserung
                        improvement = ((strategy_acc - random_acc) / random_acc) * 100 if random_acc > 0 else 0
                        
                        improvements.append({
                            'Classifier': classifier,
                            'Strategy': strategy,
                            'Budget': int(budget_pct * 100),
                            'Improvement (%)': improvement
                        })
        
        if not improvements:
            logger.warning("Keine Verbesserungsdaten gefunden!")
            return
            
        # DataFrame erstellen
        imp_df = pd.DataFrame(improvements)
        
        # Figure
        fig, ax = plt.subplots(figsize=(12, 8))
        
        # Gruppierter Barplot
        x = np.arange(len(BUDGET_PERCENTAGES))
        width = 0.15
        
        # Erstelle eine eindeutige Farbe für jede Klassifikator-Strategie-Kombination
        colors = plt.cm.tab20(np.linspace(0, 1, len(classifiers) * len(strategies)))
        color_idx = 0
        
        for i, classifier in enumerate(classifiers):
            for j, strategy in enumerate(strategies):
                data = imp_df[(imp_df['Classifier'] == classifier) & 
                             (imp_df['Strategy'] == strategy)]
                
                if not data.empty:
                    values = []
                    for b in BUDGET_PERCENTAGES:
                        budget_data = data[data['Budget'] == int(b*100)]
                        if not budget_data.empty:
                            values.append(budget_data['Improvement (%)'].values[0])
                        else:
                            values.append(0)
                    
                    offset = (i * len(strategies) + j - len(classifiers) * len(strategies) / 2) * width
                    bars = ax.bar(x + offset, values, width, 
                                 label=f'{classifier} - {strategy}',
                                 color=colors[color_idx])
                    color_idx += 1
        
        # Achsenbeschriftung
        ax.set_xlabel('Budget (%)', fontsize=12)
        ax.set_ylabel('Improvement over Random Sampling (%)', fontsize=12)
        ax.set_title('Active Learning Improvement Analysis', fontsize=14)
        ax.set_xticks(x)
        ax.set_xticklabels([f'{int(b*100)}%' for b in BUDGET_PERCENTAGES])
        
        # Nulllinie
        ax.axhline(y=0, color='black', linestyle='-', linewidth=0.5)
        
        # Legende
        ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=9)
        
        # Grid
        ax.grid(True, alpha=0.3, axis='y')
        
        plt.tight_layout()
        
        # Speichern
        filename = 'plots/active_learning_improvement_analysis.png'
        try:
            plt.savefig(filename, dpi=300, bbox_inches='tight')
            logger.info(f"[ok] Improvement-Analyse erstellt: {filename}")
        except Exception as e:
            logger.error(f"Fehler beim Speichern der Improvement-Analyse: {e}")
            
        plt.close()
        
    except Exception as e:
        logger.error(f"Fehler bei plot_improvement_analysis: {e}")
        import traceback
        traceback.print_exc()

# -------------------------------------------------------------------------------
# 13) Label-Einsparungs-Analyse Funktionen (NEU)
# -------------------------------------------------------------------------------
def calculate_label_savings(all_results, target_performance_percentages=[0.90, 0.95, 0.98]):
    """
    Berechnet die Label-Einsparung für Active Learning Strategien.
    
    Args:
        all_results: Liste aller Experiment-Ergebnisse
        target_performance_percentages: Prozentsätze der Random Sampling 100% Performance
    
    Returns:
        DataFrame mit Label-Einsparungen
    """
    savings_results = []
    
    # Gruppiere nach Klassifikator
    classifiers = sorted(list(set(r['classifier'] for r in all_results)))
    
    for classifier in classifiers:
        # Hole Random Sampling Performance bei 100% Budget als Referenz
        random_100_results = [r for r in all_results 
                            if r['classifier'] == classifier 
                            and r['strategy'] == 'Random Sampling' 
                            and r['budget_pct'] == 1.0]
        
        if not random_100_results:
            continue
            
        # Durchschnittliche Performance bei 100% Budget
        random_100_acc = np.mean([r['accuracy'] for r in random_100_results])
        
        # Für verschiedene Ziel-Performance-Level
        for target_pct in target_performance_percentages:
            target_accuracy = random_100_acc * target_pct
            
            # Für jede Strategie
            for strategy in ['Random Sampling', 'Entropy Sampling', 'Margin Sampling', 'Least Confidence']:
                strategy_results = [r for r in all_results 
                                  if r['classifier'] == classifier 
                                  and r['strategy'] == strategy]
                
                if not strategy_results:
                    continue
                
                # Aggregiere Lernkurven über alle Runs
                all_curves = []
                for r in strategy_results:
                    if 'n_labeled_list' in r and 'accuracies' in r:
                        all_curves.append((r['n_labeled_list'], r['accuracies']))
                
                if not all_curves:
                    continue
                
                # Finde minimale Label-Anzahl um Ziel-Accuracy zu erreichen
                labels_needed = []
                
                for n_labeled_list, accuracies in all_curves:
                    # Interpoliere um den Punkt zu finden, wo target_accuracy erreicht wird
                    if len(accuracies) > 0 and max(accuracies) >= target_accuracy:
                        # Finde ersten Punkt, der target_accuracy überschreitet
                        for i, acc in enumerate(accuracies):
                            if acc >= target_accuracy:
                                labels_needed.append(n_labeled_list[i])
                                break
                    else:
                        # Ziel nicht erreicht - verwende Maximum
                        labels_needed.append(60000)  # Max dataset size
                
                if labels_needed:
                    avg_labels_needed = np.mean(labels_needed)
                    std_labels_needed = np.std(labels_needed)
                    
                    # Berechne Einsparung gegenüber 100% (60000 Labels)
                    savings_pct = ((60000 - avg_labels_needed) / 60000) * 100
                    
                    # Berechne Einsparung gegenüber Random Sampling
                    if strategy != 'Random Sampling':
                        random_labels = next((s['avg_labels_needed'] for s in savings_results 
                                            if s['classifier'] == classifier 
                                            and s['strategy'] == 'Random Sampling' 
                                            and s['target_performance'] == int(target_pct*100)), 60000)
                        relative_savings_pct = ((random_labels - avg_labels_needed) / random_labels) * 100 if random_labels > 0 else 0
                    else:
                        relative_savings_pct = 0
                    
                    savings_results.append({
                        'classifier': classifier,
                        'strategy': strategy,
                        'target_performance': int(target_pct * 100),
                        'target_accuracy': target_accuracy,
                        'avg_labels_needed': avg_labels_needed,
                        'std_labels_needed': std_labels_needed,
                        'savings_pct': savings_pct,
                        'relative_savings_pct': relative_savings_pct,
                        'random_100_acc': random_100_acc
                    })
    
    return pd.DataFrame(savings_results)

def plot_label_savings(savings_df, dataset_name="MNIST"):
    """
    Visualisiert die Label-Einsparungen.
    """
    # Style setzen
    try:
        plt.style.use('seaborn-v0_8-whitegrid')
    except:
        plt.style.use('ggplot')
    
    # Figure mit Subplots
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    fig.suptitle(f'{dataset_name} - Label Savings Analysis', fontsize=16)
    
    # 1. Labels benötigt für verschiedene Performance-Level
    ax1 = axes[0, 0]
    
    # Gruppiere nach Klassifikator und Target Performance
    for classifier in savings_df['classifier'].unique():
        for target in savings_df['target_performance'].unique():
            data = savings_df[(savings_df['classifier'] == classifier) & 
                            (savings_df['target_performance'] == target)]
            
            if not data.empty:
                strategies = data['strategy'].values
                labels_needed = data['avg_labels_needed'].values
                errors = data['std_labels_needed'].values
                
                x = np.arange(len(strategies))
                width = 0.2
                offset = (target - 95) * width / 5  # Offset basierend auf target
                
                ax1.bar(x + offset, labels_needed, width, 
                       yerr=errors, capsize=5,
                       label=f'{classifier} - {target}% of baseline',
                       alpha=0.7)
    
    ax1.set_xlabel('Strategy')
    ax1.set_ylabel('Labels Needed')
    ax1.set_title('Labels Required to Reach Target Performance')
    ax1.set_xticks(np.arange(len(strategies)))
    ax1.set_xticklabels(strategies, rotation=45, ha='right')
    ax1.legend(bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=8)
    ax1.grid(axis='y', alpha=0.3)
    
    # 2. Relative Einsparung gegenüber Random Sampling
    ax2 = axes[0, 1]
    
    # Pivot für Heatmap
    pivot_savings = savings_df[savings_df['strategy'] != 'Random Sampling'].pivot_table(
        values='relative_savings_pct',
        index=['classifier', 'strategy'],
        columns='target_performance',
        fill_value=0
    )
    
    if not pivot_savings.empty and sns is not None:
        sns.heatmap(pivot_savings, 
                    annot=True, 
                    fmt='.1f', 
                    cmap='RdYlGn',
                    center=0,
                    cbar_kws={'label': 'Label Savings (%)'},
                    ax=ax2)
        ax2.set_title('Label Savings vs Random Sampling (%)')
        ax2.set_xlabel('Target Performance (% of baseline)')
    
    # 3. Absolute Label-Anzahl pro Klassifikator bei 95% Performance
    ax3 = axes[1, 0]
    
    data_95 = savings_df[savings_df['target_performance'] == 95]
    
    classifiers = data_95['classifier'].unique()
    strategies = ['Random Sampling', 'Entropy Sampling', 'Margin Sampling', 'Least Confidence']
    
    x = np.arange(len(classifiers))
    width = 0.2
    
    for i, strategy in enumerate(strategies):
        values = []
        errors = []
        for classifier in classifiers:
            row = data_95[(data_95['classifier'] == classifier) & 
                         (data_95['strategy'] == strategy)]
            if not row.empty:
                values.append(row['avg_labels_needed'].values[0])
                errors.append(row['std_labels_needed'].values[0])
            else:
                values.append(0)
                errors.append(0)
        
        ax3.bar(x + i*width - 1.5*width, values, width, 
               yerr=errors, capsize=5,
               label=strategy, alpha=0.8)
    
    ax3.set_xlabel('Classifier')
    ax3.set_ylabel('Labels Needed')
    ax3.set_title('Labels Needed to Reach 95% of Baseline Performance')
    ax3.set_xticks(x)
    ax3.set_xticklabels(classifiers, rotation=45, ha='right')
    ax3.legend()
    ax3.grid(axis='y', alpha=0.3)
    
    # Referenzlinie bei 60000 (100% der Daten)
    ax3.axhline(y=60000, color='red', linestyle='--', alpha=0.5, label='Full Dataset')
    
    # 4. Zusammenfassungstabelle
    ax4 = axes[1, 1]
    ax4.axis('tight')
    ax4.axis('off')
    
    # Erstelle Zusammenfassungstabelle für 95% Performance
    summary_data = []
    for classifier in classifiers:
        row_data = [classifier]
        for strategy in strategies:
            data = data_95[(data_95['classifier'] == classifier) & 
                          (data_95['strategy'] == strategy)]
            if not data.empty:
                labels = data['avg_labels_needed'].values[0]
                savings = data['savings_pct'].values[0]
                row_data.append(f'{int(labels):,}\n({savings:.1f}% saved)')
            else:
                row_data.append('N/A')
        summary_data.append(row_data)
    
    table = ax4.table(cellText=summary_data,
                     colLabels=['Classifier'] + strategies,
                     cellLoc='center',
                     loc='center')
    
    table.auto_set_font_size(False)
    table.set_fontsize(9)
    table.scale(1.2, 2)
    
    # Style the header row
    for i in range(len(strategies) + 1):
        table[(0, i)].set_facecolor('#40466e')
        table[(0, i)].set_text_props(weight='bold', color='white')
    
    ax4.set_title('Summary: Labels Needed for 95% Performance', pad=20)
    
    plt.tight_layout()
    
    # Speichern
    filename = f'plots/{dataset_name.lower()}_label_savings_analysis.png'
    plt.savefig(filename, dpi=300, bbox_inches='tight')
    print(f"[ok] Label-Einsparungs-Analyse erstellt: {filename}")
    
    plt.close()
    
    return savings_df

def create_label_savings_report(savings_df, dataset_name="MNIST"):
    """
    Erstellt einen detaillierten Bericht über Label-Einsparungen.
    """
    report = []
    report.append("\n" + "="*80)
    report.append(f"LABEL-EINSPARUNGS-BERICHT - {dataset_name}")
    report.append("="*80)
    
    # Für jedes Performance-Level
    for target_perf in sorted(savings_df['target_performance'].unique()):
        report.append(f"\nZIEL: {target_perf}% der Baseline-Performance")
        report.append("-"*60)
        
        target_data = savings_df[savings_df['target_performance'] == target_perf]
        
        # Nach Klassifikator gruppieren
        for classifier in sorted(target_data['classifier'].unique()):
            classifier_data = target_data[target_data['classifier'] == classifier]
            baseline_acc = classifier_data['random_100_acc'].iloc[0]
            target_acc = classifier_data['target_accuracy'].iloc[0]
            
            report.append(f"\n{classifier}:")
            report.append(f"  Baseline (Random 100%): {baseline_acc:.4f}")
            report.append(f"  Ziel-Accuracy: {target_acc:.4f}")
            report.append(f"  Labels benötigt:")
            
            # Sortiere nach Labels benötigt
            sorted_data = classifier_data.sort_values('avg_labels_needed')
            
            for _, row in sorted_data.iterrows():
                strategy = row['strategy']
                labels = row['avg_labels_needed']
                std = row['std_labels_needed']
                savings = row['savings_pct']
                rel_savings = row['relative_savings_pct']
                
                report.append(f"    - {strategy:<20}: {int(labels):>6,} ± {int(std):>4} "
                            f"({savings:>5.1f}% gespart)")
                
                if strategy != 'Random Sampling' and rel_savings > 0:
                    report.append(f"      -> {rel_savings:.1f}% weniger Labels als Random Sampling")
    
    # Beste Strategien
    report.append("\n\nBESTE STRATEGIEN (bei 95% Performance):")
    report.append("-"*60)
    
    data_95 = savings_df[savings_df['target_performance'] == 95]
    
    for classifier in sorted(data_95['classifier'].unique()):
        classifier_data = data_95[data_95['classifier'] == classifier]
        best_row = classifier_data.loc[classifier_data['avg_labels_needed'].idxmin()]
        
        report.append(f"{classifier}: {best_row['strategy']} "
                     f"(nur {int(best_row['avg_labels_needed']):,} Labels = "
                     f"{best_row['savings_pct']:.1f}% Einsparung)")
    
    # Durchschnittliche Einsparungen
    report.append("\n\nDURCHSCHNITTLICHE EINSPARUNGEN ÜBER ALLE KLASSIFIKATOREN:")
    report.append("-"*60)
    
    avg_savings = data_95.groupby('strategy')['relative_savings_pct'].mean()
    for strategy, savings in avg_savings.items():
        if strategy != 'Random Sampling':
            report.append(f"{strategy}: {savings:.1f}% weniger Labels als Random Sampling")
    
    report.append("\n" + "="*80)
    
    # Ausgabe und Speichern
    report_text = "\n".join(report)
    print(report_text)
    
    filename = f'reports/{dataset_name.lower()}_label_savings_report.txt'
    
    with open(filename, 'w', encoding='utf-8') as f:
        f.write(report_text)
    
    print(f"\n[ok] Label-Einsparungs-Bericht gespeichert: {filename}")
    
    return report_text

# -------------------------------------------------------------------------------
# Hauptprogramm
# -------------------------------------------------------------------------------
def main():
    """
    Haupteinstiegspunkt für das erweiterte Active Learning Experiment.
    """
    print("="*80)
    print("ACTIVE LEARNING AUF MNIST - ROBUSTE VERSION MIT STATISTISCHER ANALYSE")
    print("="*80)
    
    # System Info
    print(f"Python Version: {sys.version.split()[0]}")
    print(f"PyTorch Version: {torch.__version__}")
    print(f"NumPy Version: {np.__version__}")
    print(f"Pandas Version: {pd.__version__}")
    print(f"Scikit-learn Version: {sklearn.__version__}")
    print(f"SciPy Version: {scipy.__version__}")
    
    # Device Info
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    if device.type == 'cuda':
        print(f"Verwende GPU: {torch.cuda.get_device_name(0)}")
        print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")
    else:
        print("Verwende CPU (keine GPU gefunden)")
    
    print(f"\nExperiment-Konfiguration:")
    print(f"- Anzahl Runs: {N_RUNS}")
    print(f"- Budget-Stufen: {[f'{int(b*100)}%' for b in BUDGET_PERCENTAGES]}")
    print(f"- Batch-Größe: {BATCH_SIZE}")
    print(f"- Signifikanzniveau: {SIGNIFICANCE_LEVEL}")
    print("="*80)
    
    # Daten laden
    try:
        X_train_cnn, X_train_flat, y_train, X_test_cnn, X_test_flat, y_test = load_mnist_data()
    except Exception as e:
        logger.error(f"Kritischer Fehler beim Laden der Daten: {e}")
        return 1
    
    # Klassifikatoren und Strategien definieren
    classifiers = ['CNN', 'Naive Bayes', 'Random Forest']
    strategies = [
        ('Random Sampling', random_sampling),
        ('Entropy Sampling', entropy_sampling),
        ('Margin Sampling', margin_sampling),
        ('Least Confidence', least_confidence_sampling)
    ]
    
    # Experimente durchführen
    all_results = []
    total_start_time = time.time()
    
    total_experiments = len(classifiers) * len(strategies)
    current_experiment = 0
    
    for classifier_name in classifiers:
        for strategy_name, strategy_func in strategies:
            current_experiment += 1
            print(f"\n{'='*60}")
            print(f"Experiment {current_experiment}/{total_experiments}: {classifier_name} + {strategy_name}")
            print(f"{'='*60}")
            
            experiment_start_time = time.time()
            
            try:
                results = run_active_learning_experiment(
                    X_train_cnn, X_train_flat, y_train,
                    X_test_cnn, X_test_flat, y_test,
                    classifier_name, strategy_name, strategy_func,
                    BUDGET_PERCENTAGES, BATCH_SIZE
                )
                all_results.extend(results)
                
                experiment_time = time.time() - experiment_start_time
                print(f"\n[ok] {classifier_name} + {strategy_name} abgeschlossen in {experiment_time/60:.1f} Minuten")
                
            except Exception as e:
                logger.error(f"Kritischer Fehler bei {classifier_name} + {strategy_name}: {e}")
                import traceback
                traceback.print_exc()
                continue
    
    # Gesamtzeit
    total_time = time.time() - total_start_time
    print(f"\n[ok] Alle Experimente abgeschlossen in {total_time/60:.1f} Minuten")
    
    # Überprüfe ob Ergebnisse vorhanden sind
    if not all_results:
        logger.error("Keine Experimenteergebnisse vorhanden!")
        return 1
    
    # Ergebnisse in DataFrame konvertieren für statistische Analyse
    try:
        results_df = pd.DataFrame([{
            'classifier': r['classifier'],
            'strategy': r['strategy'],
            'budget_pct': r['budget_pct'],
            'run': r['run'],
            'n_labeled': r['n_labeled'],
            'accuracy': r['accuracy'],
            'f1_score': r['f1_score'],
            'avg_query_time': r.get('avg_query_time', 0),
            'avg_train_time': r.get('avg_train_time', 0)
        } for r in all_results])
    except Exception as e:
        logger.error(f"Fehler beim Erstellen des Results DataFrame: {e}")
        return 1
    
    # Statistische Analyse durchführen
    print("\n" + "="*60)
    print("Führe statistische Analyse durch...")
    print("="*60)
    
    try:
        stat_results = perform_statistical_analysis(results_df)
    except Exception as e:
        logger.error(f"Fehler bei der statistischen Analyse: {e}")
        stat_results = pd.DataFrame()
    
    # Statistischen Bericht erstellen
    try:
        statistical_report = create_statistical_report(stat_results)
    except Exception as e:
        logger.error(f"Fehler beim Erstellen des statistischen Berichts: {e}")
    
    # Label-Einsparungs-Analyse (NEU)
    print("\n" + "="*60)
    print("Berechne Label-Einsparungen...")
    print("="*60)
    
    try:
        # Berechne Label-Einsparungen für verschiedene Performance-Level
        savings_df = calculate_label_savings(all_results, target_performance_percentages=[0.90, 0.95, 0.98])
        
        # Visualisiere Label-Einsparungen
        plot_label_savings(savings_df, dataset_name="MNIST")
        
        # Erstelle detaillierten Bericht
        label_savings_report = create_label_savings_report(savings_df, dataset_name="MNIST")
        
        # Speichere als CSV
        savings_csv = 'results/mnist_label_savings.csv'
        savings_df.to_csv(savings_csv, index=False)
        print(f"[ok] Label-Einsparungen gespeichert: {savings_csv}")
        
    except Exception as e:
        logger.error(f"Fehler bei der Label-Einsparungs-Analyse: {e}")
        import traceback
        traceback.print_exc()
    
    # Ergebnisse visualisieren
    print("\n" + "="*60)
    print("Erstelle Visualisierungen...")
    print("="*60)
    
    try:
        # Pro Klassifikator mit Signifikanz
        plot_per_classifier_with_significance(all_results, stat_results)
        
        # Statistische Zusammenfassung
        plot_statistical_summary(stat_results)
        
        # Finale Vergleichsmatrix
        plot_final_comparison(all_results)
        
        # Improvement Analyse
        plot_improvement_analysis(all_results)
        
        print("[ok] Alle Visualisierungen erstellt")
    except Exception as e:
        logger.error(f"Fehler bei der Visualisierung: {e}")
        import traceback
        traceback.print_exc()
    
    # Ergebnisse speichern
    try:
        # Detaillierte Ergebnisse
        csv_filename = 'results/mnist_active_learning_results.csv'
        results_df.to_csv(csv_filename, index=False)
        print(f"\n[ok] Ergebnisse gespeichert in: {csv_filename}")
        
        # Statistische Ergebnisse
        if not stat_results.empty:
            stat_csv_filename = 'results/mnist_statistical_analysis.csv'
            stat_results.to_csv(stat_csv_filename, index=False)
            print(f"[ok] Statistische Analyse gespeichert in: {stat_csv_filename}")
        
        # Zusammenfassung als Excel (wenn verfügbar)
        if EXCEL_AVAILABLE:
            excel_filename = 'results/mnist_active_learning_summary.xlsx'
            try:
                with pd.ExcelWriter(excel_filename, engine='openpyxl') as writer:
                    # Raw results
                    results_df.to_excel(writer, sheet_name='Raw Results', index=False)
                    
                    # Statistical analysis
                    if not stat_results.empty:
                        stat_results.to_excel(writer, sheet_name='Statistical Analysis', index=False)
                    
                    # Summary by classifier and strategy
                    summary = results_df.groupby(['classifier', 'strategy', 'budget_pct'])[['accuracy', 'f1_score']].agg(['mean', 'std'])
                    summary.to_excel(writer, sheet_name='Summary Statistics')
                    
                    # Best combinations at 100% budget
                    final_results = results_df[results_df['budget_pct'] == 1.0]
                    if not final_results.empty:
                        best_combinations = final_results.groupby(['classifier', 'strategy'])[['accuracy', 'f1_score']].mean()
                        best_combinations = best_combinations.sort_values('accuracy', ascending=False)
                        best_combinations.to_excel(writer, sheet_name='Best Combinations')
                    
                    # Significant improvements
                    if not stat_results.empty and 'significant' in stat_results.columns:
                        sig_improvements = stat_results[stat_results['significant']].sort_values('cliffs_delta', ascending=False)
                        if not sig_improvements.empty:
                            sig_improvements.to_excel(writer, sheet_name='Significant Improvements', index=False)
                    
                    # Label savings (NEU)
                    if 'savings_df' in locals() and not savings_df.empty:
                        savings_df.to_excel(writer, sheet_name='Label Savings', index=False)
                
                print(f"[ok] Zusammenfassung gespeichert in: {excel_filename}")
            except Exception as e:
                logger.error(f"Fehler beim Excel-Export: {e}")
        
    except Exception as e:
        logger.error(f"Fehler beim Speichern der Ergebnisse: {e}")
    
    # Abschlusszusammenfassung
    print("\n" + "="*80)
    print("EXPERIMENT ERFOLGREICH ABGESCHLOSSEN")
    print(f"Gesamtanzahl Experimente: {len(all_results)}")
    print(f"Datensatzgröße: {len(y_train):,} Trainingsbilder")
    print(f"Klassifikatoren: {len(classifiers)}")
    print(f"Query-Strategien: {len(strategies)}")
    print(f"Budget-Stufen: {len(BUDGET_PERCENTAGES)}")
    print(f"Wiederholungen pro Experiment: {N_RUNS}")
    
    # Statistische Zusammenfassung
    if not stat_results.empty:
        total_comparisons = len(stat_results)
        significant_count = stat_results['significant'].sum() if 'significant' in stat_results.columns else 0
        print(f"\nStatistische Analyse:")
        print(f"- Anzahl Vergleiche: {total_comparisons}")
        print(f"- Signifikante Ergebnisse: {significant_count} ({significant_count/total_comparisons*100:.1f}%)")
        print(f"- Verwendeter Test: Wilcoxon Signed-Rank Test")
        print(f"- Effektstärkemaß: Cliff's Delta")
        print(f"- Multiple Vergleiche: Bonferroni-Korrektur")
    
    print("\nNEU: Label-Einsparungs-Analyse durchgeführt!")
    print("- Visualisierung: plots/mnist_label_savings_analysis.png")
    print("- Bericht: reports/mnist_label_savings_report.txt")
    print("="*80)
    
    return 0


if __name__ == "__main__":
    try:
        exit_code = main()
        sys.exit(exit_code)
    except Exception as e:
        logger.error(f"Unerwarteter Fehler im Hauptprogramm: {e}")
        import traceback
        traceback.print_exc()
        sys.exit(1)

ACTIVE LEARNING AUF MNIST - ROBUSTE VERSION MIT STATISTISCHER ANALYSE
Python Version: 3.13.5
PyTorch Version: 2.7.1+cu126
NumPy Version: 2.2.6
Pandas Version: 2.2.3
Scikit-learn Version: 1.7.1
SciPy Version: 1.16.0
Verwende GPU: NVIDIA GeForce RTX 4060 Laptop GPU
GPU Memory: 7.6 GB

Experiment-Konfiguration:
- Anzahl Runs: 3
- Budget-Stufen: ['20%', '40%', '60%', '80%', '100%']
- Batch-Größe: 500
- Signifikanzniveau: 0.05
14:11:55 [INFO] Lade vollständigen MNIST-Datensatz...
14:11:58 [INFO] [ok] Datensatz geladen: 60,000 Trainingsbilder, 10,000 Testbilder
14:11:58 [INFO]   CNN Format: (60000, 1, 28, 28) | Flat Format: (60000, 784)
14:11:58 [INFO]   Klassen: 10
14:11:58 [INFO]   Speicherbedarf: 358.9 MB

Experiment 1/12: CNN + Random Sampling
14:11:58 [INFO] 
CNN + Random Sampling - Budget: 20% (12,000 Samples)
14:11:58 [INFO]   Run 1/3
14:12:36 [INFO]     12,000 labeled -> Accuracy: 0.9888 (Train: 2.7s, Query: 0.02s)
14:12:42 [INFO]     Final: 12,000 labeled -> Accuracy: 0.9901, F1: 0.

SystemExit: 0